In [1]:
# import beautiful soup
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import tweepy
import pandas as pd
import time

In [2]:
# chrome drive
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# NASA news URL https://mars.nasa.gov/#news_and_events
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
# Scrape page with Soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#save title and paragraph and date
news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div",class_="article_teaser_body").text
news_date = soup.find("div", class_="list_date").text

print(f"Date: {news_date}")
print(f"Title: {news_title}")
print(f"Paragraphe: {news_p}")



Date: June  1, 2018
Title: NASA CubeSats Steer Toward Mars
Paragraphe: NASA has achieved a first for the class of tiny spacecraft known as CubeSats, which are opening new access to space.


In [6]:
#JPL Mars Space Images - featured image
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_img)

In [7]:
#scrape browser to find the image
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#image = soup.find('img', class_="fancybox-image")['src']
image = soup.find('article', class_="carousel_item").get('style')
#thanks kat
image = image.split("('", 1)[1].split("')")[0]
print (image)

/spaceimages/images/wallpaper/PIA18907-1920x1200.jpg


In [8]:
featured_image_url = "https://jpl.nasa.gov" + image
print (featured_image_url)

https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA18907-1920x1200.jpg


In [9]:
# # find the relative image url
# img_url_rel = soup.find('figure', class_='lede').find('img')['src']
# img_url_rel

In [10]:
# # Use the base url to create an absolute url
# img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
# img_url

In [11]:
# Mars weather twitter account to scrap the data
# import dependencies
import tweepy
# import tweeter API key authentification
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [12]:
# Twitter credentials and APi authentification
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [13]:
# find the target user
target_user = "MarsWxReport"
mars_tweet = api.user_timeline(target_user , count = 1)
latest_mars_weather=mars_tweet[0]['text']
latest_mars_weather

'RT @chrfrde: Three views of #ISS pass tonight as seen from The Hague, NL. 7 x 0.2 ms unfiltered mono in Celestron 8" at f/10. https://t.co/…'

In [14]:
# Mars fact
url_facts= "https://space-facts.com/mars/"
browser.visit(url_facts)

In [15]:
table = pd.read_html(url_facts)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
# add headers to the columns to replace 0 and 1
df_mars_table = table [0]
df_mars_table.columns = ["Mars Variables", "Data"]
# reset index to Variables
df_mars_table.set_index(["Mars Variables"])

,Data
Mars Variables,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
mars_table_html = df_mars_table.to_html()
mars_table_html = mars_table_html.replace("\n", "")
mars_table_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars Variables</th>      <th>Data</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astro

In [18]:
# Mars hemispheres
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)

In [21]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# create hemisphere list
mars_hemispheres=[]

In [22]:
# grab the different hemispheres
# looping through the pages
# use find by tag http://splinter.readthedocs.io/en/latest/finding.html
for i in range (4):
    time.sleep(2)
    hemisphere_images = browser.find_by_tag ("h3")
    hemisphere_images [i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img_product = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ img_product
# Append the dictionary with the image url string and the hemisphere title to a list.
    hemisphere_image_dict={"title":img_title,"img_url":img_url}
    mars_hemispheres.append(hemisphere_image_dict)
    browser.back()

print(mars_hemispheres)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
